In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks

# === Helper: Parse a frame and extract required keypoint coordinates ===
def extract_keypoint_y(landmarks, key_name):
    for kp in landmarks:
        if kp["name"] == key_name and kp["conf"] > 0.5:
            return kp["y"]
    return None

# === Build Combined Y-Series from JSON ===
def get_combined_y_series_from_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    y_series = []
    for frame in data:
        lm = frame["landmarks"]

        lw_y = extract_keypoint_y(lm, "left_wrist")
        rw_y = extract_keypoint_y(lm, "right_wrist")
        le_y = extract_keypoint_y(lm, "left_elbow")
        re_y = extract_keypoint_y(lm, "right_elbow")

        if None in (lw_y, rw_y, le_y, re_y):
            continue

        avg_wrist_y = (lw_y + rw_y) / 2
        avg_elbow_y = (le_y + re_y) / 2
        combined_y = (avg_wrist_y + avg_elbow_y) / 2
        y_series.append(combined_y)

    return y_series

# === Rep Counting and Visualization ===
def count_reps_and_visualize(y_series, video_name, output_plot_dir):
    if len(y_series) < 10:
        return 0

    y_series = np.array(y_series)
    y_series = (y_series - np.min(y_series)) / (np.max(y_series) - np.min(y_series))
    inverted_y = 1 - y_series
    smoothed = np.convolve(inverted_y, np.ones(5) / 5, mode='valid')

    peaks, _ = find_peaks(smoothed, height=0.6, distance=10)
    troughs, _ = find_peaks(-smoothed, height=-0.6, distance=10)
    reps = min(len(peaks), len(troughs))

    # Plot
    os.makedirs(output_plot_dir, exist_ok=True)
    plt.figure(figsize=(10, 4))
    plt.plot(smoothed, label='Smoothed Signal')
    plt.plot(peaks, smoothed[peaks], "ro", label='Peaks')
    plt.plot(troughs, smoothed[troughs], "go", label='Troughs')
    plt.title(f"{video_name} - {reps} Reps Detected")
    plt.xlabel("Frame Index")
    plt.ylabel("Normalized Y Position")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_plot_dir, f"{video_name}_rep_plot.png"))
    plt.close()

    return reps

# === Main Process (for multiple JSONs) ===
def process_json_videos(input_dir, output_csv, plot_dir):
    import csv

    results = []
    for file in os.listdir(input_dir):
        if file.endswith(".json"):
            json_path = os.path.join(input_dir, file)
            print(f"Processing: {file}...")

            try:
                y_series = get_combined_y_series_from_json(json_path)
                reps = count_reps_and_visualize(y_series, os.path.splitext(file)[0], plot_dir)
                results.append([file, reps])
            except Exception as e:
                print(f"Error processing {file}: {e}")
                results.append([file, "Error"])

    # Write to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Video_JSON", "Reps"])
        writer.writerows(results)

    print(f"\n✅ Results saved to {output_csv}")
    print(f"🖼️  Plots saved to {plot_dir}")

# === Example Usage ===
if __name__ == "__main__":
    input_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\shoulder press\Json"
    output_csv = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\shoulder press\Json\shoulder_press_counts.csv"
    plot_dir = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\shoulder press\Json"
 
    process_json_videos(input_folder, output_csv, plot_dir)


Processing: shoulder press_0_pose.json...
Processing: shoulder press_10_pose.json...
Processing: shoulder press_11_pose.json...
Processing: shoulder press_1_pose.json...

✅ Results saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\shoulder press\Json\shoulder_press_counts.csv
🖼️  Plots saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Kaggle\raw_data\data-btc\shoulder press\Json
